In [1]:
!pip install catboost
!pip install wandb

In [2]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# from sklearn.linear_model import SGDOneClassSVM
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import wandb
wandb.init(project="DACON_235876")

cv=10
seed=1011

def set_seeds(seed=seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

set_seeds()

train = pd.read_csv("/content/drive/MyDrive/DACON-Basic/235876_손동작 분류 경진대회/data/train.csv")
test = pd.read_csv("/content/drive/MyDrive/DACON-Basic/235876_손동작 분류 경진대회/data/test.csv")

train.head()

wandb: Currently logged in as: gnoeyheat (use `wandb login --relogin` to force relogin)


,id,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,...,sensor_24,sensor_25,sensor_26,sensor_27,sensor_28,sensor_29,sensor_30,sensor_31,sensor_32,target
0,1,-6.149463,-0.929714,9.058368,-7.017854,-2.958471,0.179233,-0.956591,-0.972401,5.956213,...,-7.026436,-6.006282,-6.005836,7.043084,21.884650,-3.064152,-5.247552,-6.026107,-11.990822,1
1,2,-2.238836,-1.003511,5.098079,-10.880357,-0.804562,-2.992123,26.972724,-8.900861,-5.968298,...,-1.996714,-7.933806,-3.136773,8.774211,10.944759,9.858186,-0.969241,-3.935553,-15.892421,1
2,3,19.087934,-2.092514,0.946750,-21.831788,9.119235,17.853587,-21.069954,-15.933212,-9.016039,...,-6.889685,54.052330,-6.109238,12.154595,6.095989,-40.195088,-3.958124,-8.079537,-5.160090,0
3,4,-2.211629,-1.930904,21.888406,-3.067560,-0.240634,2.985056,-29.073369,0.200774,-1.043742,...,-2.126170,-1.035526,2.178769,10.032723,-1.010897,-3.912848,-2.980338,-12.983597,-3.001077,1
4,5,3.953852,2.964892,-36.044802,0.899838,26.930210,11.004409,-21.962423,-11.950189,-20.933785,...,-2.051761,10.917567,1.905335,-13.004707,17.169552,2.105194,3.967986,11.861657,-27.088846,2


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2335 entries, 0 to 2334
Data columns (total 34 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         2335 non-null   int64  
 1   sensor_1   2335 non-null   float64
 2   sensor_2   2335 non-null   float64
 3   sensor_3   2335 non-null   float64
 4   sensor_4   2335 non-null   float64
 5   sensor_5   2335 non-null   float64
 6   sensor_6   2335 non-null   float64
 7   sensor_7   2335 non-null   float64
 8   sensor_8   2335 non-null   float64
 9   sensor_9   2335 non-null   float64
 10  sensor_10  2335 non-null   float64
 11  sensor_11  2335 non-null   float64
 12  sensor_12  2335 non-null   float64
 13  sensor_13  2335 non-null   float64
 14  sensor_14  2335 non-null   float64
 15  sensor_15  2335 non-null   float64
 16  sensor_16  2335 non-null   float64
 17  sensor_17  2335 non-null   float64
 18  sensor_18  2335 non-null   float64
 19  sensor_19  2335 non-null   float64
 20  sensor_2

In [4]:
### Valid

X = train.drop(["id", "target"], axis=1)
y = train.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed, stratify=y)

### Test

# X_train = train.drop(["id", "target"], axis=1)
# y_train = train.target
# X_test = test.drop(["id"], axis=1)

# X_train=pd.get_dummies(X_train, drop_first=True)
# X_test=pd.get_dummies(X_test, drop_first=True)

### Scaler

scaler=StandardScaler().fit(X_train)
# scaler=MinMaxScaler().fit(X_train)

X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

X_train.shape, X_test.shape

((1751, 32), (584, 32))

* Outlier Detection Feature Extraction

In [5]:
m1=OneClassSVM()
m2=IsolationForest(n_jobs=-1, random_state=seed)
m3=LocalOutlierFactor(novelty=True, n_jobs=-1)

def OD_feature(model, X_train, y_train, X_test, type="soft", scale=None):

    train_feature=[]
    test_feature=[]

    for label in np.unique(y_train):

        clf=model.fit(X_train[y_train==label])

        if type=="soft":
            train_feature_normal=pd.Series(clf.decision_function(X_train[y_train==label]), index=y_train[y_train==label].index)
            train_feature_abnormal=pd.Series(clf.decision_function(X_train[y_train!=label]), index=y_train[y_train!=label].index)
            test_feature.append(pd.Series(clf.decision_function(X_test)))
        elif type=="hard":
            train_feature_normal=pd.Series(clf.predict(X_train[y_train==label]), index=y_train[y_train==label].index)
            train_feature_abnormal=pd.Series(clf.predict(X_train[y_train!=label]), index=y_train[y_train!=label].index)
            test_feature.append(pd.Series(clf.predict(X_test)))

        train_feature.append(pd.concat([train_feature_normal, train_feature_abnormal], axis=0).sort_index())

    train_feature=np.stack(train_feature, axis=1)
    test_feature=np.stack(test_feature, axis=1)

    if scale=="standard":
        scaler=StandardScaler().fit(train_feature)
        train_feature=scaler.transform(train_feature)
        test_feature=scaler.transform(test_feature)
    elif scale=="minmax":
        scaler=MinMaxScaler().fit(train_feature)
        train_feature=scaler.transform(train_feature)
        test_feature=scaler.transform(test_feature)

    return train_feature, test_feature

In [6]:
# Outlier Detection Feature Extraction - soft
train_v1, test_v1 = OD_feature(m1, X_train, y_train, X_test, type="soft", scale="standard")
train_v2, test_v2 = OD_feature(m2, X_train, y_train, X_test, type="soft", scale="standard")
train_v3, test_v3 = OD_feature(m3, X_train, y_train, X_test, type="soft", scale="standard")

# Feature Engineering
X_train=np.concatenate((X_train, train_v1, train_v2, train_v3), axis=1)
X_test=np.concatenate((X_test, test_v1, test_v2, test_v3), axis=1)
X_train.shape, X_test.shape

In [7]:
# Outlier Detection Feature Extraction - hard
train_v1, test_v1 = OD_feature(m1, X_train, y_train, X_test, type="hard", scale=None)
train_v2, test_v2 = OD_feature(m2, X_train, y_train, X_test, type="hard", scale=None)
train_v3, test_v3 = OD_feature(m3, X_train, y_train, X_test, type="hard", scale=None)

# Feature Engineering
X_train=np.concatenate((X_train, train_v1, train_v2, train_v3), axis=1)
X_test=np.concatenate((X_test, test_v1, test_v2, test_v3), axis=1)
X_train.shape, X_test.shape

In [8]:
estimator1 = LogisticRegression(n_jobs=-1, random_state=seed)
estimator2 = GaussianNB()
estimator3 = KNeighborsClassifier(n_jobs=-1)
estimator4 = SVC(probability=True, random_state=seed)
estimator5 = RandomForestClassifier(n_jobs=-1, random_state=seed)
estimator6 = ExtraTreesClassifier(n_jobs=-1, random_state=seed)
estimator7 = AdaBoostClassifier(random_state=seed)
estimator8 = GradientBoostingClassifier(random_state=seed)
estimator9 = HistGradientBoostingClassifier(random_state=seed)
estimator10 = XGBClassifier(n_jobs=-1, random_state=seed)
estimator11 = LGBMClassifier(n_jobs=-1, random_state=seed)
estimator12 = CatBoostClassifier(verbose=False, random_state=seed)
# estimator13 = MLPClassifier(random_state=seed)

In [9]:
def get_stacking_ml_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
    
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
    
    train_fold_pred = np.zeros((X_train_n.shape[0], 1))
    test_pred = np.zeros((X_test_n.shape[0], n_folds))
    
    for folder_counter, (train_index, valid_index) in enumerate(skf.split(X_train_n, y_train_n)):
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]
        
        model.fit(X_tr, y_tr)
        train_fold_pred[valid_index, :] = model.predict_proba(X_te)[:,1].reshape(-1,1)
        test_pred[:, folder_counter] = model.predict_proba(X_test_n)[:,1]
        
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    return train_fold_pred, test_pred_mean

In [10]:
%%time

base_ml = [estimator1, estimator2, estimator3, estimator4, estimator5,
           estimator6, estimator7, estimator8, estimator9, estimator10,
           estimator11, estimator12]

meta_ml_X_train=[]
meta_ml_X_test=[]
for estimator in base_ml:
    temp_X_train, temp_X_test = get_stacking_ml_datasets(estimator, X_train, y_train.values, X_test, cv)
    meta_ml_X_train.append(temp_X_train)
    meta_ml_X_test.append(temp_X_test)
    
meta_ml_X_train=np.hstack(meta_ml_X_train)
meta_ml_X_test=np.hstack(meta_ml_X_test)

meta_ml_X_train.shape, meta_ml_X_test.shape

CPU times: user 13min, sys: 12.3 s, total: 13min 12s
Wall time: 7min 48s


In [11]:
# Outlier Detection Feature Extraction - stacking
train_v1, test_v1 = OD_feature(m1, meta_ml_X_train, y_train, meta_ml_X_test, type="soft", scale="minmax")
train_v2, test_v2 = OD_feature(m2, meta_ml_X_train, y_train, meta_ml_X_test, type="soft", scale="minmax")
train_v3, test_v3 = OD_feature(m3, meta_ml_X_train, y_train, meta_ml_X_test, type="soft", scale="minmax")

# Feature Engineering
meta_ml_X_train=np.concatenate((meta_ml_X_train, train_v1, train_v2, train_v3), axis=1)
meta_ml_X_test=np.concatenate((meta_ml_X_test, test_v1, test_v2, test_v3), axis=1)
meta_ml_X_train.shape, meta_ml_X_test.shape

((1751, 24), (584, 24))

In [12]:
meta_clf=LogisticRegression(n_jobs=-1, random_state=seed)
meta_clf.fit(meta_ml_X_train, y_train)
submission=meta_clf.predict(meta_ml_X_test)

## Valid

def ACCURACY(true, pred):
    score = np.mean(true==pred)
    return score

accuracy=ACCURACY(y_test, submission)
print(accuracy)

wandb.log({'accuracy': accuracy, 'cv': cv, 'seed': seed})

### Test

# sample_submission = pd.read_csv("/content/drive/MyDrive/DACON-Basic/235876_손동작 분류 경진대회/data/sample_submission.csv")
# sample_submission.target = submission
# sample_submission.to_csv("/content/drive/MyDrive/DACON-Basic/235876_손동작 분류 경진대회/submission.csv",index=False)

0.6883561643835616
